In [9]:
import os
import json
import cv2
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm

# ==============================================
# YOLO 모델 로드 + 최적화
# ==============================================
model = YOLO("yolov8s-pose.pt")
model.to('cuda')

# COCO keypoint name
kp_names = [
    "nose","left_eye","right_eye","left_ear","right_ear",
    "left_shoulder","right_shoulder","left_elbow","right_elbow",
    "left_wrist","right_wrist","left_hip","right_hip",
    "left_knee","right_knee","left_ankle","right_ankle"
]

# ==============================================
def convert_frame(original_frame, original_total=600, new_total=100):
    return int(original_frame / original_total * new_total)

def load_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# ==============================================
# JSON 전체 경로 인덱싱
# ==============================================
def index_all_json(json_root):
    json_map = {}
    for root, _, files in os.walk(json_root):
        for f in files:
            if f.lower().endswith(".json"):
                key = os.path.splitext(f)[0]
                json_map[key] = os.path.join(root, f)
    return json_map


# ==============================================
# Skeleton batch 추출 + 이전 프레임 보정
# ==============================================
def extract_skeleton(video_path):

    cap = cv2.VideoCapture(video_path)
    batch_size = 64
    frame_data = []
    batch = []

    prev = [None] * 51
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        batch.append(frame)

        if len(batch) == batch_size:
            results = model(batch, verbose=False)
            for r in results:
                if r.keypoints is not None and len(r.keypoints) > 0:
                    keypoints = r.keypoints.xy[0].cpu().numpy()
                    conf = r.keypoints.conf[0].cpu().numpy()
                    sk = []
                    for i in range(17):
                        x, y = keypoints[i]
                        c = conf[i]
                        sk += [float(x), float(y), float(c)]
                    prev = sk
                else:
                    sk = prev
                frame_data.append((frame_idx, sk))
                frame_idx += 1

            batch = []

    if len(batch) > 0:
        results = model(batch, verbose=False)
        for r in results:
            if r.keypoints is not None and len(r.keypoints) > 0:
                keypoints = r.keypoints.xy[0].cpu().numpy()
                conf = r.keypoints.conf[0].cpu().numpy()
                sk = []
                for i in range(17):
                    x, y = keypoints[i]
                    c = conf[i]
                    sk += [float(x), float(y), float(c)]
                prev = sk
            else:
                sk = prev
            frame_data.append((frame_idx, sk))
            frame_idx += 1

    cap.release()
    return frame_data

# ==============================================
# 영상 1개 처리
# ==============================================
def process_video(video_path, json_map):

    file_name = os.path.basename(video_path)
    key = os.path.splitext(file_name)[0]

    frames = extract_skeleton(video_path)

    # ------------------------------------------
    # N case (낙상 없음)
    # ------------------------------------------
    if "_N_" in file_name:
        return [[file_name, frame, 0] + sk for frame, sk in frames]

    # ------------------------------------------
    # Y case (낙상 있음)
    # ------------------------------------------
    if "_SY_" in file_name:

        if key not in json_map:
            print("[WARN] JSON 없음:", file_name)
            return None

        json_path = json_map[key]
        meta = load_json(json_path)

        start600 = meta["sensordata"]["fall_start_frame"]
        end600   = meta["sensordata"]["fall_end_frame"]

        start100 = convert_frame(start600)
        end100   = convert_frame(end600)

        rows = []
        for frame_idx, sk in frames:
            label = 1 if start100 <= frame_idx <= end100 else 0
            rows.append([file_name, frame_idx, label] + sk)

        return rows

    print("[WARN] 파일명에 Y/N 없음:", file_name)
    return None


# ==============================================
# 전체 폴더 처리 (재귀)
# ==============================================
def process_all(video_root, json_root, save_path):

    json_map = index_all_json(json_root)

    rows = []

    video_list = []
    for root, _, files in os.walk(video_root):
        for f in files:
            if f.lower().endswith((".mp4",".avi",".mov")):
                video_list.append(os.path.join(root, f))

    for video_path in tqdm(video_list, desc="전체 영상 처리중"):
        result = process_video(video_path, json_map)
        if result is not None:
            rows += result

    col_file = ["file"]
    col_frame = ["frame"]
    col_label = ["label"]

    col_skel = []
    for kp in kp_names:
        col_skel += [f"{kp}_x", f"{kp}_y", f"{kp}_score"]

    df = pd.DataFrame(rows, columns = col_file + col_frame + col_label + col_skel)

    df.to_csv(save_path, index=False, encoding='utf-8-sig')

    print("\n🔥 완료:", save_path)

process_all(
    video_root="/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/01.원천데이터/TS/영상",
    json_root="/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/02.라벨링데이터/TL/영상",
    save_path="/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/output.csv"
)

전체 영상 처리중: 100%|██████████| 4128/4128 [4:07:13<00:00,  3.59s/it]



🔥 완료: /content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/output.csv


In [11]:
train_df = pd.read_csv('/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/output_train.csv')
train_df.head()

,file,frame,label,nose_x,nose_y,nose_score,left_eye_x,left_eye_y,left_eye_score,right_eye_x,...,left_knee_score,right_knee_x,right_knee_y,right_knee_score,left_ankle_x,left_ankle_y,left_ankle_score,right_ankle_x,right_ankle_y,right_ankle_score
0,02077_H_A_N_C1.mp4,0,0,433.201904,137.339722,0.123687,433.536316,133.542877,0.128915,435.075867,...,0.992070,471.505096,269.176941,0.983481,459.491669,322.209106,0.983306,476.900970,311.007446,0.971570
1,02077_H_A_N_C1.mp4,1,0,432.706665,139.028076,0.119133,432.644653,135.262482,0.107487,434.446259,...,0.990311,471.737030,269.089661,0.984615,457.475311,320.276947,0.979955,477.392151,310.526978,0.972040
2,02077_H_A_N_C1.mp4,2,0,432.811005,140.320831,0.082605,432.059906,136.109604,0.069186,433.391357,...,0.989685,470.083954,268.875702,0.983896,456.548462,320.828400,0.976981,473.975281,310.356018,0.968025
3,02077_H_A_N_C1.mp4,3,0,431.418213,141.639893,0.084149,430.199036,137.462433,0.066609,431.576538,...,0.990412,470.245453,267.599304,0.986282,456.372253,320.752991,0.978816,474.933746,308.751160,0.972351
4,02077_H_A_N_C1.mp4,4,0,425.485809,142.749344,0.093535,424.375854,138.612152,0.070890,425.632202,...,0.989743,471.258270,266.028259,0.987877,456.317413,321.318939,0.978681,476.850220,307.429901,0.975985


In [14]:
train_df.loc[train_df['label'] == 0, 'label'].count(), train_df.loc[train_df['label'] == 1, 'label'].count()

(np.int64(394576), np.int64(20424))

In [20]:
import os
import json
import cv2
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm

# ==============================================
# YOLO 모델 로드 + 최적화
# ==============================================
model = YOLO("yolov8s-pose.pt")
model.to('cuda')

# COCO keypoint name
kp_names = [
    "nose","left_eye","right_eye","left_ear","right_ear",
    "left_shoulder","right_shoulder","left_elbow","right_elbow",
    "left_wrist","right_wrist","left_hip","right_hip",
    "left_knee","right_knee","left_ankle","right_ankle"
]

# ==============================================
def convert_frame(original_frame, original_total=600, new_total=100):
    return int(original_frame / original_total * new_total)

def load_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# ==============================================
# JSON 전체 경로 인덱싱
# ==============================================
def index_all_json(json_root):
    json_map = {}
    for root, _, files in os.walk(json_root):
        for f in files:
            if f.lower().endswith(".json"):
                key = os.path.splitext(f)[0]
                json_map[key] = os.path.join(root, f)
    return json_map


# ==============================================
# Skeleton batch 추출 + 이전 프레임 보정
# ==============================================
def extract_skeleton(video_path):

    cap = cv2.VideoCapture(video_path)
    batch_size = 64
    frame_data = []
    batch = []

    prev = [None] * 51
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        batch.append(frame)

        if len(batch) == batch_size:
            results = model(batch, verbose=False)
            for r in results:
                if r.keypoints is not None and len(r.keypoints) > 0:
                    keypoints = r.keypoints.xy[0].cpu().numpy()
                    conf = r.keypoints.conf[0].cpu().numpy()
                    sk = []
                    for i in range(17):
                        x, y = keypoints[i]
                        c = conf[i]
                        sk += [float(x), float(y), float(c)]
                    prev = sk
                else:
                    sk = prev
                frame_data.append((frame_idx, sk))
                frame_idx += 1

            batch = []

    if len(batch) > 0:
        results = model(batch, verbose=False)
        for r in results:
            if r.keypoints is not None and len(r.keypoints) > 0:
                keypoints = r.keypoints.xy[0].cpu().numpy()
                conf = r.keypoints.conf[0].cpu().numpy()
                sk = []
                for i in range(17):
                    x, y = keypoints[i]
                    c = conf[i]
                    sk += [float(x), float(y), float(c)]
                prev = sk
            else:
                sk = prev
            frame_data.append((frame_idx, sk))
            frame_idx += 1

    cap.release()
    return frame_data

# ==============================================
# 영상 1개 처리
# ==============================================
def process_video(video_path, json_map):

    file_name = os.path.basename(video_path)
    key = os.path.splitext(file_name)[0]

    frames = extract_skeleton(video_path)

    # ------------------------------------------
    # N case (낙상 없음)
    # ------------------------------------------
    if "_N_" in file_name:
        return [[file_name, frame, 0] + sk for frame, sk in frames]

    # ------------------------------------------
    # Y case (낙상 있음)
    # ------------------------------------------
    if "_SY_" in file_name:

        if key not in json_map:
            print("[WARN] JSON 없음:", file_name)
            return None

        json_path = json_map[key]
        meta = load_json(json_path)

        start600 = meta["sensordata"]["fall_start_frame"]
        end600   = meta["sensordata"]["fall_end_frame"]

        start100 = convert_frame(start600)
        end100   = convert_frame(end600)

        rows = []
        for frame_idx, sk in frames:
            label = 1 if start100 <= frame_idx <= end100 else 0
            rows.append([file_name, frame_idx, label] + sk)

        return rows

    print("[WARN] 파일명에 Y/N 없음:", file_name)
    return None


# ==============================================
# 전체 폴더 처리 (재귀)
# ==============================================
def process_all(video_root, json_root, save_path):

    json_map = index_all_json(json_root)

    rows = []

    video_list = []
    for root, _, files in os.walk(video_root):
        for f in files:
            if f.lower().endswith((".mp4",".avi",".mov")):
                video_list.append(os.path.join(root, f))

    for video_path in tqdm(video_list, desc="전체 영상 처리중"):
        result = process_video(video_path, json_map)
        if result is not None:
            rows += result

    col_file = ["file"]
    col_frame = ["frame"]
    col_label = ["label"]

    col_skel = []
    for kp in kp_names:
        col_skel += [f"{kp}_x", f"{kp}_y", f"{kp}_score"]

    df = pd.DataFrame(rows, columns = col_file + col_frame + col_label + col_skel)

    df.to_csv(save_path, index=False, encoding='utf-8-sig')

    print("\n🔥 완료:", save_path)

process_all(
    video_root="/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/01.원천데이터/VS/영상",
    json_root="/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/02.라벨링데이터/VL/영상",
    save_path="/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/output_val.csv"
)

전체 영상 처리중:   4%|▍         | 23/596 [01:36<35:53,  3.76s/it]

[WARN] JSON 없음: 00026_H_A_SY_C4_fall_detected.mp4


전체 영상 처리중:  11%|█         | 67/596 [04:35<33:20,  3.78s/it]

[WARN] JSON 없음: 00105_H_A_SY_C2_fall_detected.mp4


전체 영상 처리중:  23%|██▎       | 135/596 [09:16<25:55,  3.37s/it]

[WARN] JSON 없음: 00711_H_D_SY_C6_fall_detected.mp4


전체 영상 처리중:  25%|██▌       | 151/596 [10:19<27:58,  3.77s/it]

[WARN] JSON 없음: 02047_H_A_SY_C5_fall_detected.mp4


전체 영상 처리중: 100%|██████████| 596/596 [37:14<00:00,  3.75s/it]



🔥 완료: /content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/output_val.csv


In [21]:
val_df = pd.read_csv('/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/output_val.csv')
val_df.head()

,file,frame,label,nose_x,nose_y,nose_score,left_eye_x,left_eye_y,left_eye_score,right_eye_x,...,left_knee_score,right_knee_x,right_knee_y,right_knee_score,left_ankle_x,left_ankle_y,left_ankle_score,right_ankle_x,right_ankle_y,right_ankle_score
0,02366_H_A_SY_C7.mp4,0,0,713.038818,227.906616,0.888511,717.906677,223.113281,0.924195,712.661621,...,0.977322,726.436951,385.376892,0.880025,731.935120,442.980225,0.940621,733.062866,437.023102,0.809121
1,02366_H_A_SY_C7.mp4,1,0,713.864014,227.872955,0.891030,718.779541,223.278336,0.923748,713.481201,...,0.977421,723.235352,385.598083,0.885203,732.559082,442.147339,0.941275,729.728516,437.006561,0.816043
2,02366_H_A_SY_C7.mp4,2,0,713.848267,228.159546,0.878212,718.976013,223.683899,0.919033,714.304138,...,0.973288,724.934143,384.681549,0.852213,729.944092,440.952271,0.927905,731.427673,435.665894,0.765373
3,02366_H_A_SY_C7.mp4,3,0,714.660950,228.523041,0.877201,720.220764,224.102509,0.916390,714.938904,...,0.973639,726.083923,385.064911,0.850723,730.536621,440.957397,0.929609,731.798218,435.709595,0.765879
4,02366_H_A_SY_C7.mp4,4,0,714.568359,229.067261,0.866399,720.067688,224.768997,0.911367,715.088623,...,0.971766,726.226807,386.860168,0.835380,729.281128,442.915710,0.924214,731.149902,437.982208,0.744962


In [22]:
val_df.loc[val_df['label'] == 0, 'label'].count(), val_df.loc[val_df['label'] == 1, 'label'].count()

(np.int64(56128), np.int64(3072))